In [17]:
import json
import re
import glob
import os

from tqdm import tqdm
import pandas as pd
from langdetect import detect as lang_detect
from banglakit import lemmatizer as lem
from banglakit.lemmatizer import BengaliLemmatizer

lemmatizer = BengaliLemmatizer()

In [18]:
!pip install git+https://github.com/banglakit/lemmatizer.git#egg=banglakit-lemmatizer

In [19]:
input_dir = 'output'
output_dir = 'output/Sentences/'

input_files = glob.glob(os.path.join(output_dir, '1-*.csv'))
combined_df= pd.concat([pd.read_csv(file)[["text","source"]] for file in input_files])
combined_df.shape

(2711772, 2)

In [20]:
for file in input_files:
    print(pd.read_csv(file).shape)

(80771, 4)
(98099, 2)
(23208, 2)
(81953, 2)
(2420356, 4)
(7385, 2)


In [21]:
def is_all_bangla(text):
    if bool(re.match("^[\u0980-\u09FF ।,?!.]+$",text)):  ## only valid characters
        return True
    return False

def lemmatize_sentence(text):
    result=""
    for word in text.split():
        lem_word= lemmatizer.lemmatize(word, pos=lem.POS_NOUN)
        if is_all_bangla(lem_word):
            result+= lem_word + " "
        else:
            result+= word + " "
    return result


def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [24]:
combined_sentences= list(combined_df["text"])
combined_sentences[:5]

['আমার রবি সিম কোন ক্যাটাগরিতে পড়ে',
 'সবচাইতে কম কত টাকায় ১জিবি কিনতে পারবো',
 'বড় বড় রাঘব বোয়ালরা আরও গুরুতর অপরাধ করেও পার পেয়ে যায়',
 ' আর সামান্য অপরাধের কারনে এমন সিদ্দ্বান্ত নেওয়া ঠিক হয়নি',
 'লোকমানের ত্যাগের প্রতি এটা অবিচার করা হয়েছে']

In [28]:
########## Random Shuffling
from random import shuffle
shuffle(combined_sentences)
combined_sentences[:5]

['আমি ছাই বস মাশরাপির নামে একটা  স্টুডিয়াম বনানো হক ু',
 'অনুসরণ করলে সে তাকে নির্লজ্জতা ও',
 'ওনি নিজেকে কি মোনে করে জানিনা',
 'আহ্.মন ভরে গেল,কিন্তু রাইটারের নাম দেখে অন্তরে রক্তক্ষরণ শুরু হয়ে গেছে. :/',
 'কিনতু তাতে কি তোমার ব্যাবসায়  তো লাভ হইছে']

In [33]:
########## selecting 500k for next steps
total=500000
sentence_list= combined_sentences[:total]

In [34]:
######### Sentence index 0 to 100k
lemmatized_sentences= []

for text in tqdm(sentence_list[:total]):
    lemmatized_sentences.append(lemmatize_sentence(text.strip()))


100%|██████████| 500000/500000 [00:20<00:00, 24335.54it/s]

In [35]:
lemmatized_sentences[:10]

['আমি ছাই বস মাশরাপি নাম এক স্টুডিয়াম বনানো হক ু ',
 'অনুসরণ কর সে তাকে নির্লজ্জতা ও ',
 'ওনি নিজে কি মোন কর জানিনা ',
 'আহ্.মন ভর গেল,কিন্তু রাইটার নাম দেখ অন্তর রক্তক্ষরণ শুরু হয়ে গেছে. :/ ',
 'কিনতু তাত কি তোমার ব্যাবসায় তো লাভ হইছ ',
 'যারা সালমা কে দোষ দিয় না জেন কমেন্ট করছো তাদের মা বোন যেন সালমা র চাইতেও অতি কষ্ট থা ',
 'মিথিলা অবাক দৃষ্টিত তার দি তাকিয় বল “এত বছর ধর ওখান চেক দিচ্ছেন ',
 'বরগুনা মিন্নি পক্ষ থেকে মিথিলা প্রাণঢালা শুভেচ্ছা, এবং সাথ দোয়া রইলো আরো কিছু পিক ভাইরাল জন্য ',
 'আর বাঙ্গালী জাতি ভাষা এটাতো বুঝলাম না ',
 'তাই জলপাই বাহিনী পাঠানো হোক ']

In [36]:
sentence_list[:10]

['আমি ছাই বস মাশরাপির নামে একটা  স্টুডিয়াম বনানো হক ু',
 'অনুসরণ করলে সে তাকে নির্লজ্জতা ও',
 'ওনি নিজেকে কি মোনে করে জানিনা',
 'আহ্.মন ভরে গেল,কিন্তু রাইটারের নাম দেখে অন্তরে রক্তক্ষরণ শুরু হয়ে গেছে. :/',
 'কিনতু তাতে কি তোমার ব্যাবসায়  তো লাভ হইছে',
 'যারা সালমা কে দোষ দিয়ে না জেনে কমেন্ট করছো তাদের মা বোন যেন সালমা র চাইতেও অতি কষ্টে থাকে',
 'মিথিলা অবাক দৃষ্টিতে তার দিকে তাকিয়ে বলল “এত বছর ধরে ওখানে চেক দিচ্ছেন',
 'বরগুনা মিন্নির পক্ষে থেকে মিথিলাকে প্রাণঢালা শুভেচ্ছা, এবং সাথে দোয়া রইলো আরো কিছু পিক ভাইরালের জন্য  ',
 ' আর বাঙ্গালী জাতির ভাষা এটাতো বুঝলাম না',
 ' তাই জলপাই বাহিনী পাঠানো হোক ']

In [37]:
print(len(sentence_list))
print(len(lemmatized_sentences))

500000
500000


In [ ]:
final_sentences= []
for clean_idx,sentence in tqdm(enumerate(sentence_list[50001:100000])):
    is_unique=True
    l1= lemmatize_sentence(sentence)
    for lem_idx,l2 in enumerate(lemmatized_sentences):
        if clean_idx==lem_idx:
            continue
        if jaccard_similarity(l1.split(),l2.split()) > .7:
#             print(l1+"\n"+l1+"\nDiscarded")
            is_unique=False
            break
    
    if is_unique:
        final_sentences.append(sentence)





0it [00:00, ?it/s]



1it [00:00,  7.55it/s]



2it [00:00,  7.50it/s]



3it [00:00,  7.06it/s]



4it [00:00,  7.12it/s]



5it [00:00,  7.04it/s]



6it [00:00,  6.77it/s]



7it [00:01,  6.55it/s]



9it [00:01,  7.81it/s]



10it [00:01,  7.61it/s]



11it [00:01,  7.35it/s]



12it [00:01,  7.27it/s]



13it [00:01,  7.28it/s]



14it [00:01,  6.75it/s]



15it [00:02,  5.78it/s]



16it [00:02,  5.92it/s]



17it [00:02,  5.82it/s]



18it [00:02,  6.10it/s]



19it [00:02,  6.33it/s]



20it [00:02,  6.19it/s]



21it [00:03,  5.84it/s]



22it [00:03,  5.33it/s]



23it [00:03,  5.65it/s]



24it [00:03,  6.09it/s]



25it [00:03,  6.29it/s]



26it [00:03,  6.66it/s]



27it [00:04,  6.51it/s]



29it [00:04,  7.63it/s]



30it [00:04,  7.14it/s]



31it [00:04,  6.79it/s]



32it [00:04,  6.33it/s]



33it [00:04,  6.56it/s]



34it [00:05,  6.84it/s]



35it [00:05,  6.44it/s]



37it [00:06,  3.24it/s]



38it [00:07,  1.63it/s]



39it [00:08,  2.09it/s]



40it [00:0

221it [00:39,  5.69it/s]



222it [00:39,  6.04it/s]



223it [00:40,  6.01it/s]



224it [00:40,  6.03it/s]



225it [00:40,  6.34it/s]



226it [00:40,  6.40it/s]



227it [00:40,  6.56it/s]



228it [00:40,  6.63it/s]



229it [00:40,  5.80it/s]



230it [00:41,  5.95it/s]



231it [00:41,  6.12it/s]



232it [00:41,  6.38it/s]



233it [00:41,  6.61it/s]



234it [00:41,  6.63it/s]



235it [00:41,  6.04it/s]



236it [00:42,  5.76it/s]



237it [00:42,  6.16it/s]



238it [00:42,  6.47it/s]



240it [00:42,  7.11it/s]



241it [00:42,  6.57it/s]



242it [00:42,  6.85it/s]



243it [00:43,  7.03it/s]



244it [00:43,  5.92it/s]



245it [00:43,  6.19it/s]



246it [00:43,  6.51it/s]



247it [00:43,  6.84it/s]



248it [00:43,  6.66it/s]



249it [00:43,  6.92it/s]



250it [00:44,  6.91it/s]



251it [00:44,  6.88it/s]



252it [00:44,  6.85it/s]



253it [00:46,  1.51it/s]



254it [00:46,  1.96it/s]



255it [00:46,  2.47it/s]



256it [00:46,  3.10it/s]



257it [00:46,  3.45i

432it [01:35,  2.58it/s]



433it [01:35,  3.17it/s]



434it [01:36,  3.77it/s]



435it [01:36,  4.43it/s]



436it [01:36,  4.78it/s]



437it [01:36,  5.20it/s]



438it [01:36,  5.59it/s]



439it [01:36,  6.03it/s]



441it [01:36,  7.30it/s]



442it [01:37,  7.23it/s]



443it [01:37,  6.64it/s]



444it [01:37,  6.61it/s]



445it [01:37,  6.78it/s]



446it [01:37,  6.53it/s]



447it [01:37,  6.72it/s]



448it [01:37,  6.88it/s]



449it [01:38,  6.76it/s]



450it [01:38,  5.79it/s]



451it [01:38,  5.85it/s]



452it [01:38,  6.19it/s]



453it [01:38,  6.36it/s]



454it [01:39,  5.94it/s]



455it [01:39,  6.21it/s]



456it [01:39,  6.45it/s]



457it [01:39,  6.68it/s]



458it [01:39,  6.81it/s]



459it [01:39,  6.65it/s]



460it [01:39,  6.92it/s]



461it [01:40,  6.70it/s]



462it [01:40,  6.89it/s]



463it [01:40,  6.83it/s]



464it [01:40,  6.28it/s]



465it [01:40,  5.96it/s]



466it [01:40,  6.33it/s]



467it [01:40,  6.63it/s]



468it [01:41,  6.75i

645it [02:24,  6.02it/s]



646it [02:24,  6.10it/s]



647it [02:24,  6.25it/s]



648it [02:24,  6.15it/s]



649it [02:25,  6.02it/s]



650it [02:25,  5.86it/s]



651it [02:25,  6.17it/s]



652it [02:25,  5.52it/s]



653it [02:25,  5.98it/s]



654it [02:26,  5.93it/s]



655it [02:27,  1.51it/s]



656it [02:27,  1.96it/s]



657it [02:28,  2.47it/s]



658it [02:28,  2.73it/s]



659it [02:28,  3.35it/s]



660it [02:28,  3.82it/s]



661it [02:28,  4.20it/s]



662it [02:29,  4.81it/s]



663it [02:31,  1.30it/s]



664it [02:31,  1.55it/s]



665it [02:31,  1.99it/s]



666it [02:31,  2.52it/s]



667it [02:31,  3.08it/s]



670it [02:32,  4.05it/s]



672it [02:32,  5.00it/s]



673it [02:32,  5.36it/s]



674it [02:32,  5.70it/s]



676it [02:32,  6.44it/s]



677it [02:33,  6.06it/s]



678it [02:33,  6.19it/s]



679it [02:34,  2.03it/s]



680it [02:34,  2.51it/s]



681it [02:34,  3.03it/s]



682it [02:34,  3.62it/s]



683it [02:35,  4.12it/s]



684it [02:35,  4.73i

865it [03:12,  6.18it/s]



866it [03:12,  6.14it/s]



867it [03:12,  6.20it/s]



868it [03:12,  6.25it/s]



869it [03:12,  5.13it/s]



870it [03:12,  5.49it/s]



871it [03:13,  5.94it/s]



872it [03:13,  5.97it/s]



873it [03:13,  6.26it/s]



874it [03:13,  5.46it/s]



875it [03:13,  5.63it/s]



876it [03:13,  6.04it/s]



877it [03:14,  5.81it/s]



878it [03:14,  6.16it/s]



879it [03:14,  6.47it/s]



880it [03:14,  6.17it/s]



881it [03:14,  6.17it/s]



882it [03:14,  6.16it/s]



883it [03:15,  5.36it/s]



884it [03:15,  5.67it/s]



885it [03:15,  5.86it/s]



886it [03:15,  6.03it/s]



887it [03:15,  6.32it/s]



888it [03:17,  1.91it/s]



889it [03:17,  2.41it/s]



890it [03:17,  2.82it/s]



891it [03:17,  3.14it/s]



892it [03:17,  3.79it/s]



893it [03:18,  4.42it/s]



894it [03:18,  4.54it/s]



895it [03:18,  4.95it/s]



897it [03:18,  6.00it/s]



898it [03:18,  6.07it/s]



899it [03:23,  1.54s/it]



901it [03:23,  1.10s/it]



902it [03:23,  1.22i

...

10489it [44:00,  3.89it/s]



10490it [44:00,  4.21it/s]



10491it [44:00,  4.54it/s]



10492it [44:04,  1.42s/it]



10493it [44:05,  1.04s/it]



10494it [44:05,  1.28it/s]



10495it [44:05,  1.66it/s]



10496it [44:05,  2.05it/s]



10497it [44:05,  2.47it/s]



10499it [44:07,  1.86it/s]



10500it [44:07,  2.16it/s]



10501it [44:09,  1.12it/s]



10502it [44:09,  1.48it/s]



10503it [44:10,  1.91it/s]



10504it [44:11,  1.25it/s]



10505it [44:11,  1.60it/s]



10506it [44:11,  1.99it/s]



10507it [44:12,  2.49it/s]



10508it [44:12,  3.01it/s]



10509it [44:12,  3.33it/s]



10510it [44:12,  3.61it/s]



10511it [44:12,  4.10it/s]



10513it [44:13,  5.09it/s]



10514it [44:13,  5.32it/s]



10515it [44:13,  5.50it/s]



10516it [44:13,  5.62it/s]



10517it [44:13,  5.72it/s]



10518it [44:13,  5.62it/s]



10519it [44:14,  5.68it/s]



10520it [44:14,  5.59it/s]



10521it [44:14,  5.77it/s]



10522it [44:14,  5.80it/s]



10523it [44:14,  5.67it/s]



10524it [4

10694it [44:59,  4.18it/s]



10695it [44:59,  4.63it/s]



10696it [44:59,  4.94it/s]



10697it [44:59,  4.86it/s]



10698it [45:00,  4.93it/s]



10699it [45:00,  5.03it/s]



10700it [45:00,  4.97it/s]



10701it [45:00,  5.21it/s]



10702it [45:00,  5.26it/s]



10703it [45:01,  4.62it/s]



10704it [45:01,  4.92it/s]



10705it [45:01,  5.26it/s]



10706it [45:01,  5.20it/s]



10707it [45:01,  5.48it/s]



10708it [45:01,  5.48it/s]



10709it [45:02,  5.59it/s]



10710it [45:02,  5.63it/s]



10711it [45:02,  5.79it/s]



10712it [45:02,  5.72it/s]



10713it [45:02,  6.46it/s]



10714it [45:02,  6.27it/s]



10715it [45:03,  6.19it/s]



10716it [45:03,  5.64it/s]



10717it [45:03,  5.75it/s]



10718it [45:03,  5.58it/s]



10719it [45:03,  5.59it/s]



10720it [45:04,  5.11it/s]



10721it [45:04,  5.18it/s]



10722it [45:05,  1.85it/s]



10723it [45:05,  2.33it/s]



10724it [45:06,  2.56it/s]



10725it [45:06,  2.97it/s]



10726it [45:06,  3.44it/s]



10727it [4

10898it [45:50,  5.04it/s]



10899it [45:51,  5.19it/s]



10900it [45:51,  5.29it/s]



10901it [45:51,  5.49it/s]



10902it [45:51,  5.71it/s]



10903it [45:51,  5.75it/s]



10904it [45:52,  5.09it/s]



10905it [45:53,  1.49it/s]



10906it [45:54,  1.93it/s]



10907it [45:54,  2.22it/s]



10908it [45:54,  2.68it/s]



10909it [45:54,  3.23it/s]



10910it [45:54,  3.69it/s]



10911it [45:55,  4.05it/s]



10913it [45:55,  4.95it/s]



10914it [45:55,  4.97it/s]



10915it [45:55,  5.25it/s]



10916it [45:55,  5.42it/s]



10917it [45:56,  5.02it/s]



10918it [45:56,  5.21it/s]



10919it [45:56,  5.33it/s]



10920it [45:56,  5.36it/s]



10921it [45:56,  5.52it/s]



10922it [45:56,  5.44it/s]



10923it [45:57,  4.98it/s]



10924it [45:58,  1.66it/s]



10925it [45:58,  2.11it/s]



10926it [45:59,  2.59it/s]



10927it [45:59,  3.03it/s]



10928it [45:59,  3.55it/s]



10929it [45:59,  3.99it/s]



10931it [45:59,  4.42it/s]



10932it [46:00,  4.79it/s]



10933it [4

11099it [46:34,  6.41it/s]



11100it [46:34,  6.32it/s]



11101it [46:34,  6.07it/s]



11102it [46:35,  5.68it/s]



11103it [46:35,  5.87it/s]



11104it [46:35,  5.92it/s]



11105it [46:35,  5.75it/s]



11106it [46:35,  5.77it/s]



11107it [46:36,  5.78it/s]



11108it [46:36,  5.77it/s]



11109it [46:36,  5.90it/s]



11110it [46:36,  5.44it/s]



11111it [46:36,  5.53it/s]



11112it [46:36,  5.61it/s]



11113it [46:37,  5.44it/s]



11114it [46:37,  5.62it/s]



11115it [46:37,  5.76it/s]



11116it [46:37,  5.80it/s]



11117it [46:37,  5.72it/s]



11118it [46:37,  5.55it/s]



11119it [46:38,  5.59it/s]



11120it [46:38,  5.58it/s]



11121it [46:39,  1.70it/s]



11122it [46:40,  2.11it/s]



11123it [46:40,  2.61it/s]



11124it [46:40,  2.98it/s]



11125it [46:40,  3.52it/s]



11126it [46:40,  3.97it/s]



11127it [46:41,  4.31it/s]



11128it [46:42,  1.81it/s]



11129it [46:42,  2.24it/s]



11130it [46:42,  2.77it/s]



11131it [46:42,  3.25it/s]



11132it [4

11307it [47:27,  5.44it/s]



11308it [47:28,  5.66it/s]



11309it [47:28,  5.76it/s]



11310it [47:28,  5.62it/s]



11311it [47:28,  5.29it/s]



11312it [47:28,  5.38it/s]



11313it [47:29,  5.35it/s]



11314it [47:29,  5.55it/s]



11315it [47:29,  5.58it/s]



11316it [47:29,  5.66it/s]



11317it [47:29,  5.76it/s]



11318it [47:29,  5.69it/s]



11319it [47:30,  5.77it/s]



11320it [47:30,  4.78it/s]



11321it [47:30,  4.77it/s]



11322it [47:30,  5.06it/s]



11323it [47:30,  5.35it/s]



11324it [47:31,  5.50it/s]



11325it [47:31,  5.32it/s]



11326it [47:33,  1.08it/s]



11327it [47:34,  1.39it/s]



11328it [47:34,  1.80it/s]



11329it [47:36,  1.10s/it]



11330it [47:36,  1.22it/s]



11331it [47:37,  1.58it/s]



11332it [47:37,  1.97it/s]



11333it [47:37,  2.36it/s]



11334it [47:37,  2.73it/s]



11335it [47:37,  3.11it/s]



11336it [47:38,  3.62it/s]



11337it [47:38,  3.96it/s]



11338it [47:38,  4.12it/s]



11339it [47:38,  4.53it/s]



11340it [4

In [ ]:
len(final_sentences)

In [ ]:
pd.DataFrame(final_sentences[:20000]).to_csv("output/Sentences/2-combined-50kto100k.csv")

# Paragraph